In [65]:
!pip install nltk

In [87]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import Word2Vec

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [67]:
df_faq = pd.read_excel('/content/drive/MyDrive/МФТИ/ВКРС/data/faq.xlsx')
df_faq.head()

,Тема,Вопрос,Схема ответа,Кнопка ссылки на вики,Ссылка на wiki,Тэги для поиска
0,Личный кабинет,Как зарегистрироваться в ЛК?,1. Для регистрации в личном кабинете перейдите...,Регистрация в ЛК,https://wiki.greendata.store/%D0%A0%D0%B5%D0%B...,"Личный кабинет, регистрация, регистрация ЛК SA..."
1,Личный кабинет,Зачем указывать страну? |Зачем выбрирать язык?...,Кнопка с флагом в правом верхнем углу нужна дл...,NaN,NaN,"Личный кабинет, язык, не тот язык, настройка я..."
2,Личный кабинет,Не приходит подтверждение на почту,I. Если Вам не приходит письмо для подтвержден...,NaN,NaN,"Личный кабинет, регистрация, подтверждение рег..."
3,Личный кабинет,Нет почты/забыл почту. Есть ли другой канал дл...,В настоящее время доступна регистрация только ...,NaN,NaN,"Личный кабинет, регистрация, регистрация ЛК SA..."
4,Личный кабинет,Почему у меня все отображается не на том языке?,Выбор языка зависит от настроек браузера. \nCh...,NaN,NaN,"Личный кабинет, язык, не тот язык, настройка я..."


In [68]:
df_chat = pd.read_csv('/content/drive/MyDrive/МФТИ/ВКРС/data/data.csv')
df_chat = df_chat[df_chat['from'] == 'client']
df_chat.head()

,chat_id,chat_type,type,from,client_id,events_count,text,chat.last_message.from,chat.last_message.type,chat.last_message.text,chat.proactive_message.text,latest_chat_id,created_date
1,1096,missed,message,client,543,2,Здравствуйте!,client,message,Практика,Здравствуйте! Чем могу помочь? 👋,1096,2023-11-01 12:14:29
2,1096,missed,message,client,543,2,Практика,client,message,Практика,Здравствуйте! Чем могу помочь? 👋,1096,2023-11-01 12:14:37
4,1094,incoming,message,client,542,2,"Добрый день. Хотел бы узнать, есть ли у вас св...",agent,email,Добрый день. Контакты отдела кадров и актуальн...,NaN,1094,2023-10-31 06:51:26
7,894,incoming,message,client,451,6,Добрый день,agent,email,Приложение Ссылка на приложение_guid_8dc8bd4c-...,NaN,1093,2023-06-13 04:07:33
8,894,incoming,message,client,451,6,"не могу зайти в приложение, выходит ошибка",agent,email,Приложение Ссылка на приложение_guid_8dc8bd4c-...,NaN,1093,2023-06-13 04:08:03


In [69]:
df_chats = pd.read_csv('/content/drive/MyDrive/МФТИ/ВКРС/data/chats.csv')
df_client = df_chats[df_chats['last_message_from'] == 'client'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']
df_agent = df_chats[df_chats['last_message_from'] == 'agent'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']
df_client.head()

<ipython-input-69-6fe0147cb83b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_client = df_chats[df_chats['last_message_from'] == 'client'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']
<ipython-input-69-6fe0147cb83b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_client = df_chats[df_chats['last_message_from'] == 'client'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']
<ipython-input-69-6fe0147cb83b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_agent = df_chats[df_chats['last_message_from'] == 'agent'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']
<ipython-input-69-6fe0147cb83b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_agent = df_chats[df_chats['last_message_from'] == 'agent'][df_chats['chat_type']!='missed'][df_chats['chat_type']!='banned']


,chat_type,messages_count,start,finish,last_message_type,last_message_from,last_message_text,hour,weekday,month,duration
0,incoming,7,2021-07-22 14:07:03,2021-07-23 03:22:12,message,client,благодарю!,14,3,7,0 days 13:15:09
7,incoming,7,2021-08-13 08:11:54,2021-08-13 10:57:58,message,client,благодарю!,8,4,8,0 days 02:46:04
8,incoming,4,2021-08-13 08:12:44,2021-08-13 10:57:58,message,client,спасибо!,8,4,8,0 days 02:45:14
17,outgoing,2,2021-11-01 12:14:40,2021-11-01 12:26:02,message,client,⚠ http: 403 Forbidden,12,0,11,0 days 00:11:22
18,outgoing,0,2021-11-02 08:25:49,2021-11-02 08:25:49,message,client,⚠ http: 403 Forbidden,8,1,11,0 days 00:00:00


Предварительная обработка текста


In [85]:
# Предварительная обработка текста
def preprocess_text(text):
    # Удаление лишних символов и цифр
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Токенизация текста
    tokens = word_tokenize(text)
    # Приведение слов к нижнему регистру
    tokens = [word.lower() for word in tokens]
    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))  # Загрузка списка стоп-слов для русского языка
    tokens = [word for word in tokens if word not in stop_words]
    # Лемматизация слов
    lemmatizer = WordNetLemmatizer()  # Создание объекта лемматизатора
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Объединение токенов обратно в строку
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [88]:
  # Применение предобработки к столбцу с текстом
df_chat['preprocessed_text'] = df_chat['text'].apply(preprocess_text)
df_faq['preprocessed_text'] = df_faq['Вопрос'].apply(preprocess_text)
df_client['preprocessed_text'] = df_client['last_message_text'].apply(preprocess_text)
df_agent['preprocessed_text'] = df_agent['last_message_text'].apply(preprocess_text)

In [89]:
df_chat.head()

,chat_id,chat_type,type,from,client_id,events_count,text,chat.last_message.from,chat.last_message.type,chat.last_message.text,chat.proactive_message.text,latest_chat_id,created_date,preprocessed_text,is_faq_TFIDF,is_faq_Word2Vec
1,1096,missed,message,client,543,2,Здравствуйте!,client,message,Практика,Здравствуйте! Чем могу помочь? 👋,1096,2023-11-01 12:14:29,здравствуйте,False,False
2,1096,missed,message,client,543,2,Практика,client,message,Практика,Здравствуйте! Чем могу помочь? 👋,1096,2023-11-01 12:14:37,практика,False,False
4,1094,incoming,message,client,542,2,"Добрый день. Хотел бы узнать, есть ли у вас св...",agent,email,Добрый день. Контакты отдела кадров и актуальн...,NaN,1094,2023-10-31 06:51:26,добрый день хотел узнать свободные вакансии оп...,False,True
7,894,incoming,message,client,451,6,Добрый день,agent,email,Приложение Ссылка на приложение_guid_8dc8bd4c-...,NaN,1093,2023-06-13 04:07:33,добрый день,False,False
8,894,incoming,message,client,451,6,"не могу зайти в приложение, выходит ошибка",agent,email,Приложение Ссылка на приложение_guid_8dc8bd4c-...,NaN,1093,2023-06-13 04:08:03,могу зайти приложение выходит ошибка,True,True


#Количество диалогов, в которых проблема пользователя была решена



In [73]:
# Определим список слов благодарности или ключевых фраз
keywords = ['спасибо', 'благодарю', 'очень признателен', 'все отлично', 'помогли']

In [74]:
# Создадим функцию для поиска наличия слов благодарности в тексте сообщения
def contains_thanks(text):
    for keyword in keywords:
        if re.search(r'\b' + keyword + r'\b', text, re.IGNORECASE):
            return True
    return False

In [101]:
# Применим функцию к каждому сообщению пользователя и создадим новый столбец с результатами
df_client['thanks'] = df_client['last_message_text'].apply(contains_thanks)

In [100]:
# Подсчитайте количество сообщений, в которых обнаружены слова благодарности
thanks_count = df_client['thanks'].sum()

print("Количество сообщений от пользователя с благодарностью:", thanks_count)

Количество сообщений от пользователя с благодарностью: 166


In [77]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 0 to 843
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   chat_type          215 non-null    object
 1   messages_count     215 non-null    int64 
 2   start              215 non-null    object
 3   finish             215 non-null    object
 4   last_message_type  215 non-null    object
 5   last_message_from  215 non-null    object
 6   last_message_text  215 non-null    object
 7   hour               215 non-null    int64 
 8   weekday            215 non-null    int64 
 9   month              215 non-null    int64 
 10  duration           215 non-null    object
 11  preprocessed_text  215 non-null    object
 12  thanks             215 non-null    bool  
dtypes: bool(1), int64(4), object(8)
memory usage: 30.1+ KB


In [78]:
df_agent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 1 to 865
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   chat_type          479 non-null    object
 1   messages_count     479 non-null    int64 
 2   start              479 non-null    object
 3   finish             479 non-null    object
 4   last_message_type  479 non-null    object
 5   last_message_from  479 non-null    object
 6   last_message_text  479 non-null    object
 7   hour               479 non-null    int64 
 8   weekday            479 non-null    int64 
 9   month              479 non-null    int64 
 10  duration           479 non-null    object
 11  preprocessed_text  479 non-null    object
dtypes: int64(4), object(8)
memory usage: 48.6+ KB


#Количество ЧЗД

Метод TF-IDF векторизации

In [90]:
# Создаем TfidfVectorizer
vectorizer = TfidfVectorizer()

# Объединяем тексты часто задаваемых вопросов и тексты чатов для обучения векторизатора
texts = df_faq['preprocessed_text'].tolist() + df_chat['preprocessed_text'].tolist()

# Обучаем векторизатор на объединенных текстах
vectorizer.fit(texts)

# Преобразуем тексты из обеих таблиц в TF-IDF векторы
faq_vectors = vectorizer.transform(df_faq['preprocessed_text'])
chat_vectors = vectorizer.transform(df_chat['preprocessed_text'])

# Вычисляем косинусное сходство между TF-IDF векторами часто задаваемых вопросов и текстами чатов
similarity_scores = cosine_similarity(chat_vectors, faq_vectors)

# Определяем пороговое значение сходства
threshold = 0.3

# Проставляем метки часто задаваемых вопросов для каждого текста чата
df_chat['is_faq_TFIDF'] = [any(similarity_scores[i] >= threshold) for i in range(len(df_chat))]

In [91]:
df_chat['is_faq_TFIDF'].value_counts(normalize=True)

is_faq_TFIDF
False    0.915222
True     0.084778
Name: proportion, dtype: float64

In [81]:
df_chat[['text', 'is_faq_TFIDF']].head()

,text,is_faq_TFIDF
1,Здравствуйте!,False
2,Практика,False
4,"Добрый день. Хотел бы узнать, есть ли у вас св...",False
7,Добрый день,False
8,"не могу зайти в приложение, выходит ошибка",True


Метод Word2Vec

In [97]:
# Объединяем тексты часто задаваемых вопросов и тексты чатов для обучения модели Word2Vec
texts = [text.split() for text in df_faq['preprocessed_text']] + [text.split() for text in df_chat['preprocessed_text']]

# Обучаем модель Word2Vec на объединенных текстах
model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

# Функция для вычисления максимального сходства между векторами слов в тексте и векторами слов в часто задаваемых вопросах
def compute_similarity(text, faq_texts):
    similarities = []
    for faq_text in faq_texts:
        valid_words = [word for word in text if word in model.wv and word in faq_text]
        if not valid_words:
            similarities.append(0.0)
            continue
        text_vector = np.mean([model.wv[word] for word in valid_words], axis=0)
        faq_vector = np.mean([model.wv[word] for word in valid_words], axis=0)
        similarity = np.dot(text_vector, faq_vector) / (np.linalg.norm(text_vector) * np.linalg.norm(faq_vector))
        similarities.append(similarity)
    return max(similarities)

# Определяем пороговое значение сходства
threshold = 0.5

# Проставляем метки часто задаваемых вопросов для каждого текста чата
df_chat['is_faq_Word2Vec'] = [compute_similarity(text.split(), [faq_text.split() for faq_text in df_faq['preprocessed_text']]) >= threshold for text in df_chat['preprocessed_text']]

In [98]:
df_chat['is_faq_Word2Vec'].value_counts(normalize=True)

is_faq_Word2Vec
False    0.525995
True     0.474005
Name: proportion, dtype: float64

In [96]:
df_chat[['preprocessed_text', 'is_faq_Word2Vec']].head(20)

,preprocessed_text,is_faq_Word2Vec
1,здравствуйте,False
2,практика,False
4,добрый день хотел узнать свободные вакансии оп...,False
7,добрый день,False
8,могу зайти приложение выходит ошибка,True
10,подскажите корзину очистить,True
11,вики инфо восстановить удалить,True
14,подозреваю стенде изза корзины место кончилось,True
17,здравствуйте,False
18,ссылка jivo media_guid_cdeddbbcdc,False
